<a href="https://colab.research.google.com/github/ejdogar/NLP/blob/main/With_Sentences_RAG_Assignment_4_Ejaz_ul_Haq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Atomcamp/Assignments/Assignment-4/Pak Suzuki Car Reviews.csv")
dataset.head()

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,"Nov 30, 2023",\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,"Nov 29, 2023","\nA good looking car with best fuel ? economy,...",5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,"Nov 26, 2023",\nfor me i would must buy because its will sav...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,"Nov 12, 2023",\nI have used it for more than 5 years. litera...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,"Nov 09, 2023",\nExterior: 10/10 on design but the quality us...,5,5,5,4,5


# Step 1: Data Preprocessing

#### 1.1 Converting Date Column to DateTime Format

In [ ]:
dataset["Date"] = pd.to_datetime(dataset["Date"]).dt.date
dataset.head()

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,2023-11-30,\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,2023-11-29,"\nA good looking car with best fuel ? economy,...",5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,2023-11-26,\nfor me i would must buy because its will sav...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,2023-11-12,\nI have used it for more than 5 years. litera...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,2023-11-09,\nExterior: 10/10 on design but the quality us...,5,5,5,4,5


In [ ]:
dataset["Date"][0]

datetime.date(2023, 11, 30)

#### 1.2 Creating a Year Column
Splitting the Model Column into Model Name and Year Column, in order to enhance model answering capability.

In [ ]:
# dataset["Year"], dataset["Model"] = dataset["Model"].str.split(" ").str[0], dataset["Model"].str.split(" ").str[1:].str.join(" ")
# dataset.head()

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money,Year
0,0,Suzuki Bolan,1,2023-11-30,\nis gadi k baray me 3 lafaz kahunga 1 bakwaaa...,1,1,1,1,1,2023
1,1,Suzuki Swift GLX CVT,4,2023-11-29,"\nA good looking car with best fuel ? economy,...",5,4,5,4,5,2022
2,2,Suzuki Cultus Auto Gear Shift,5,2023-11-26,\nfor me i would must buy because its will sav...,5,5,5,5,5,2022
3,3,Suzuki Liana RXi (CNG),5,2023-11-12,\nI have used it for more than 5 years. litera...,5,5,5,5,5,2007
4,4,Suzuki Alto VX,4,2023-11-09,\nExterior: 10/10 on design but the quality us...,5,5,5,4,5,2022


#### 1.3 Formatting our reviews

In [ ]:
dataset["Review"] = dataset["Review"].str.replace("\n", "")
dataset.head()

,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,2023-11-30,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,2023-11-29,"A good looking car with best fuel ? economy,Dr...",5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,2023-11-26,for me i would must buy because its will save ...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,2023-11-12,I have used it for more than 5 years. literall...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,2023-11-09,Exterior: 10/10 on design but the quality used...,5,5,5,4,5


In [ ]:
import re
for index, review in enumerate(dataset['Review']):
  review = re.sub("[?.,$]", " ", review)
  dataset["Review"][index] = review
dataset.head()


<ipython-input-124-2636c59a88a7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Review"][index] = review


,Unnamed: 0,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,0,2023 Suzuki Bolan,1,2023-11-30,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1
1,1,2022 Suzuki Swift GLX CVT,4,2023-11-29,A good looking car with best fuel economy Dr...,5,4,5,4,5
2,2,2022 Suzuki Cultus Auto Gear Shift,5,2023-11-26,for me i would must buy because its will save ...,5,5,5,5,5
3,3,2007 Suzuki Liana RXi (CNG),5,2023-11-12,I have used it for more than 5 years literall...,5,5,5,5,5
4,4,2022 Suzuki Alto VX,4,2023-11-09,Exterior: 10/10 on design but the quality used...,5,5,5,4,5


#### 1.4 Dropping Unnamed Column
As this column just have indexes and is of no significance as we have system indexing available for out dataset.

In [ ]:
dataset.drop("Unnamed: 0", axis=1, inplace=True)
dataset.head()

,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,2023 Suzuki Bolan,1,2023-11-30,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,1,1,1,1,1
1,2022 Suzuki Swift GLX CVT,4,2023-11-29,A good looking car with best fuel economy Dr...,5,4,5,4,5
2,2022 Suzuki Cultus Auto Gear Shift,5,2023-11-26,for me i would must buy because its will save ...,5,5,5,5,5
3,2007 Suzuki Liana RXi (CNG),5,2023-11-12,I have used it for more than 5 years literall...,5,5,5,5,5
4,2022 Suzuki Alto VX,4,2023-11-09,Exterior: 10/10 on design but the quality used...,5,5,5,4,5


1.5 Coverting Rating from Numbers to Wrod Representation

In [ ]:
rating_dic = {1: "Worst", 2: "Bad", 3: "Average", 4: "Good", 5: "Excellent"}
dataset[["Overall_rating", "Style", "Comfort", "Fuel_Economy", "Performance", "Value_for_Money"]] = dataset[["Overall_rating",  "Style", "Comfort", "Fuel_Economy", "Performance", "Value_for_Money"]].applymap(rating_dic.get)
dataset.head()

,Model,Overall_rating,Date,Review,Style,Comfort,Fuel_Economy,Performance,Value_for_Money
0,2023 Suzuki Bolan,Worst,2023-11-30,is gadi k baray me 3 lafaz kahunga 1 bakwaaas ...,Worst,Worst,Worst,Worst,Worst
1,2022 Suzuki Swift GLX CVT,Good,2023-11-29,A good looking car with best fuel economy Dr...,Excellent,Good,Excellent,Good,Excellent
2,2022 Suzuki Cultus Auto Gear Shift,Excellent,2023-11-26,for me i would must buy because its will save ...,Excellent,Excellent,Excellent,Excellent,Excellent
3,2007 Suzuki Liana RXi (CNG),Excellent,2023-11-12,I have used it for more than 5 years literall...,Excellent,Excellent,Excellent,Excellent,Excellent
4,2022 Suzuki Alto VX,Good,2023-11-09,Exterior: 10/10 on design but the quality used...,Excellent,Excellent,Excellent,Good,Excellent


In [ ]:
review_maxlength = dataset["Review"].str.len().max()
review_maxlength

3359

In [ ]:
review_minlength = dataset["Review"].str.len().mean()
review_minlength

551.9872611464968

#### 1.6 Converting Dataframe into Sentences
Here I am converting all the data available in the dataframe into sentences, where each row would be represented by a unique sentence. Doing this will help our model learn better semantics.

In [ ]:
sentences = []
for index, row in dataset.iterrows():
  sentences.append(f"On {row[2]} someone reviewed {row[0]} with an over all rating of {row[1]} and rated Style, Comfort, Fuel Economy, Performance, Value for Money as {row[4]}, {row[5]}, {row[6]}, {row[7]}, {row[8]} respectively. Review from this customer was {row[3]}")
print(len(sentences))
print(sentences[0])

1099
On 2023-11-30 someone reviewed 2023 Suzuki Bolan with an over all rating of Worst and rated Style, Comfort, Fuel Economy, Performance, Value for Money as Worst, Worst, Worst, Worst, Worst respectively. Review from this customer was is gadi k baray me 3 lafaz kahunga 1 bakwaaas 2 bakwaaaaaaas 3 bakwaaaassaaaaaaaaasfazool tareen or gharya tareen gadi shame on Suzuki company dunya me har cheez badal gai comfort level kahan se kahan ponch gya par ye farsoooda gadi he koi isko lyne ka sochy bhi na


In [ ]:
# sentences_len = [len(sentence) for sentence in sentences]

# min_len = min(sentences_len)
# max_len = max(sentences_len)
# avg_len = sum(sentences_len) / len(sentences_len)

# print(f"Min sentence length: {min_len}")
# print(f"Max sentence length: {max_len}")
# print(f"Average sentence length: {avg_len}")

In [ ]:
# from typing import Sequence
# sentences = [sentence[:int(avg_len)] for sentence in sentences]
# for idx, sentence in enumerate(sentences):
#   if len(sentence) < int(avg_len):
#     while len(sentence) < int(avg_len):
#       sentences[idx] += " "
# print(len(sentences[0]))
# print(len(sentences[10]))
# print(len(sentences[101]))

In [ ]:
dataframe = pd.DataFrame(sentences, columns=["Review"])
dataframe.head()

,Review
0,On 2023-11-30 someone reviewed 2023 Suzuki Bol...
1,On 2023-11-29 someone reviewed 2022 Suzuki Swi...
2,On 2023-11-26 someone reviewed 2022 Suzuki Cul...
3,On 2023-11-12 someone reviewed 2007 Suzuki Lia...
4,On 2023-11-09 someone reviewed 2022 Suzuki Alt...


### Importing Necessary Libraries For Our Model

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.30.2 langchain chromadb --quiet
!pip install tiktoken --quiet
!pip install sentence_transformers lark auto-gptq optimum --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-gptq 0.6.0 requires accelerate>=0.22.0, but you have accelerate 0.21.0 which is incompatible.
auto-gptq 0.6.0 requires peft>=0.5.0, but you have peft 0.4.0 which is incompatible.
auto-gptq 0.6.0 requires transformers>=4.31.0, but you have transformers 4.30.2 which is incompatible.


# Step 2: Creating Data Chunk From our CSV File

In [ ]:
from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(dataframe, page_content_column="Review")
documents = loader.load()
documents[0].page_content

'On 2023-11-30 someone reviewed 2023 Suzuki Bolan with an over all rating of Worst and rated Style, Comfort, Fuel Economy, Performance, Value for Money as Worst, Worst, Worst, Worst, Worst respectively. Review from this customer was is gadi k baray me 3 lafaz kahunga 1 bakwaaas 2 bakwaaaaaaas 3 bakwaaaassaaaaaaaaasfazool tareen or gharya tareen gadi shame on Suzuki company dunya me har cheez badal gai comfort level kahan se kahan ponch gya par ye farsoooda gadi he koi isko lyne ka sochy bhi na'

# Step 3: Creating Embeddings of Our Chunks
Generating Embaddings based on our above created chunks.

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "google/flan-t5-base"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


In [ ]:
# !pip install chromadb

In [ ]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(documents, embeddings, persist_directory = "./llama-db")

In [ ]:
import torch
import transformers
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, GPTQConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/llama-2-7b-Chat-GPTQ")
tokenizer.pad_token = tokenizer.eos_token
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7b-Chat-GPTQ",
                              quantization_config=quantization_config_loading,
                              device_map="auto"
                        )

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [ ]:
from transformers import pipeline
import torch
from langchain.llms import HuggingFacePipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 256,
                do_sample=True,
                top_k=1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.2
                )

llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
print(llm(prompt="What are the reviews regarding Suzuki Liana Eminent Automatic in year 2023?"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



Average Rating of Suzuki Liana Eminent Automatic in Year 2023 is 4.5 out of 5. Check average rating and opinions of Suzuki Liana Eminent Automatic car models in Pakistan, along with detailed specifications, features, prices & photos. Find out complete details about Suzuki Liana Eminent Automatic technical specification, mileage, safety features, review videos and latest news.


In [ ]:
from langchain.chains import RetrievalQA

rag = RetrievalQA.from_chain_type(
        llm=llm, chain_type='stuff',
        retriever=db.as_retriever()
        )

In [ ]:
print(rag("What are the reviews regarding Suzuki Liana Eminent Automatic in year 2023?")["result"])

 The reviewer did not leave any review about Suzuki Liana Eminent Automatic in year 2023 because there is no such review available.
